### **_This was updated on 3/29/24 to use nix-library-raw, 1.1.4_**
the library is being split into 3 component libraries for future use.

#### tl;dr

A simple test of a library providing json->html data viewing, with browser or inline Razor page, using mocked data.

Solves: effective storage and viewing of divergent (1:n) data sets.

# Json support (Nix-Library-Raw)

This set of code was used to develop a support library to simplify data exploration.

It focuses on **json** data, specifically json Arrays or Objects such as would be expected from a database query using `for json path`. It wraps the conversion of the json string to an html string suitable for presentation.

Because in discovery we are frequently trying to understand relationships and cardinality, it is not usually necessary to retrieve vast quantities of data, rather this allows the display of "several" samples. Nested relationships are enclosed and can be limited to arbitrarily small samples.

A css style sheet provides the underlying styling: this can be altered for different purposes.
Some variables are used, which also allows their modification through Javascript.


## Summary of library functionality

The code is dependent on System.Text.Json (which replaces the older Newtonsoft.Json)

Functionality is wrapped mainly in the `Nest` class. This is a simple class with a `Name`, `Value`, `Nesting` and Children which ate themselves `Nest`s.

### Constructors

For general use, the Nest class is not needed, the static methods handle this for you. However if you need to use your own hierarchical structure, have at it:

`Nest(string Name, string json)` Creates a Nest from a json string representing an array or object.
`Nest(string Name, JsonNode node, int depth = 0)` Is used internally and recursively by the simpler constructor, but can be used to process a JsonObject directly.

The `Nest` class is a simpler and more compact representation than a JsonObject, so can be used to digest and store hierarchial objects for multiple uses.

For one-off rendering, the static methods below are more effective.

### Class Methods

- `string AsHtml(NestOptions options)` returns Html for displaying the object directly.
- `IEnumerable<string>Elements()` returns an enumerable series of strings describing the structure of the data (typically for understanding structure).
- `string AllElements(bool withLineBreaks = false)` returns all the elements with or without line feeds.

### Static methods

- `string Prettify(string json)` formats a json string as indented, fo human consumption.

- `string RenderHtml(string json)` return html suitable for embedding in a page using external css.
- `string RenderPage(string json, NestOptions options)` returns html with css suitable for display as a single page.

- `string GetCss(NestOptions options)` returns the css code for the specified options.

and for convenience:

- `void ToFile(string data, string path)` pushes a string to a local file.
- `string FromFile(string path)` reads a string from a local file.


### Nest Options

To fine-tune the behavior, it is possible to tweak several settings.- 

- `MaximumPixelWidth` sets the width of the enclosing container, to force horizontal scrolling, default 800.
- `TitleColumnWidth` sets the width of the leading column used to label elements, default 120;
- `SpecifyFont` defaults to true, which sets the font to consolas, monospace.
- `SmallerTitles` defaults to true, which reduces the size of titles.
- `ItalicTitles` defaults to true which makes titles italic.
- `ItemLimit` limits the number of items shown in lists, default 10.
- `DarkMode` uses a dark theme, which is the default.



## How about a serious test?

We will use Bogus to generate a slew of information, which we will reduce to json then use to test variations of options.

In [1]:
// load packages
#i "file:///n:/nix-nuget/"
// Library implementing the Nest class.
#r "nuget:nix-library-raw"  
// Used to generate random data for testing
#r "nuget:bogus"            
// Needed by the library and/or jsoin serialization
#r "nuget:System.Text.Json" 
// This allows you to use Razor Pages directly (see later example)
#r "nuget:RazorInteractive" 

Restore sources file:///n:/nix-nuget/ Installed Packages bogus, 35.5.0 nix-library-raw, 1.1.4 RazorInteractive, 1.2.0 System.Text.Json, 8.0.3

Loading extensions from `C:\Users\NickMajor\.nuget\packages\razorinteractive\1.2.0\lib\net8.0\RazorInteractive.dll`

Renders the code block as Razor markup in dotnet-interactive notebooks. 
 This extension adds a new kernel that can render Razor markdown. 
 All C# and F# variables are available in the @Model property.

In [2]:
// make some bogus data
// // This will make a string "json" that has a hierarchial object.
using Bogus;
// Some ranf-ges
int lo = 15;
int hi = 25;

// Let's start with a hierarchial object with some nested arrays.
public class DataSet
{
    public string Client { get; set; }
    public int Size { get; set; }
    public int LocationCount => Locations.Count(); 
    public IEnumerable<Location> Locations { get; set; } = new List<Location>();
}
public class Location
{
    public string LocationName { get; set; }
    public IEnumerable<string> TelephoneNumbers { get; set; } = new List<string>();
    public IEnumerable<string> AddressRows { get; set; } = new List<string>();
        public int EmployeeCount => Employees.Count();
        public IEnumerable<Employee> Employees { get; set; } = new List<Employee>();
}
public class Employee
{
    public int Id { get; set; }
    public string Name { get; set; }
    public string Title { get; set; }
    public string Email { get; set; }
    public string Telephone { get; set; }
}

var count = new Faker().Random.Number(lo, hi);
int identity = 1000;
var dataFaker = new Faker<DataSet>()
    .RuleFor(d => d.Client, f => f.Company.CompanyName())
    .RuleFor(d => d.Size, count)
    .RuleFor(d => d.Locations, f => f.Make(count, () => new Location
    {
        LocationName = f.Address.City(),
        TelephoneNumbers = f.Make(lo, () => f.Phone.PhoneNumber()),
        AddressRows = f.Make(lo, () => f.Address.StreetAddress()),
        Employees = f.Make(f.Random.Number(lo, hi), () => new Employee
        {
            Id = identity++,
            Name = f.Name.FullName(),
            Title = f.Name.JobTitle(),
            Email = f.Internet.Email(),
            Telephone = f.Phone.PhoneNumber()
        })
    }));

    // we make some data...
    var objectData = dataFaker.Generate();
    var arrayData = objectData.Locations;
    // convert the data to a displayable json string
    var objectJson = System.Text.Json.JsonSerializer.Serialize(objectData);
    var arrayJson = System.Text.Json.JsonSerializer.Serialize(arrayData);
    
   // Console.WriteLine("*** JSON OBJECT ***");
   // Nest.Prettify(objectJson).Display();
   // Console.WriteLine("*** JSON ARRAY ***");
   // Nest.Prettify(arrayJson).Display();

## Now the raw library has been added as a Nuget package.

Let's test against that!


In [3]:
using Nix.Library.Raw;
// some options:
var options = new Nest.NestOptions() { DarkMode = true, MaximumListPixelWidth = 1000 };
// generate a page to display the object:
Nest.ToFile(Nest.RenderPageScrolling(objectJson, 5, "A", options) , "./.html/Test-objectX.html");
// generate a page to generate the array:
Nest.ToFile(Nest.RenderPageScrolling(arrayJson, 5, "B", options ) , "./.html/Test-arrayX.html");

In [4]:
#!pwsh
# The two created html files can be launched in the browser dircetly in a powershell script:
./.html/Test-objectX.html
./.html/Test-arrayX.html

In [5]:
// Or we could use Razor:
#!csharp

// Write a local css file to sytle things:
Nest.ToFile(Nest.GetCss(options), "./local-test.css");
// generate a couple of json strings, one for the object, one for the array:
var A = Nest.RenderHtmlScrolling(objectJson, 5);
var B = Nest.RenderHtmlScrolling(arrayJson, 5);

// display both in a razor page directly: (the @Model exposes any variables form NetInteractive)
#!razor
<html>
  <head>
    <link rel="stylesheet" href="./local-test.css">
  </head>
  <body>
    <h3>Object rendered raw in-line</h3>
    @Raw(@Model.A)
    <h3>Array rendered raw in-line</h3>
    @Raw(@Model.B)
    <hr>
  </body>
</html>

Object rendered raw in-line 
 Client Lynch - Altenwerth 
 Size 22 
 LocationCount 22 
 Locations 
 1/5 2/5 3/5 4/5 5/5 LocationName Champlinville New Bernadettemouth West Randallfort New Montetown New Macie TelephoneNumbers 
 777.433.4105 362.291.0745 1-649-577-0681 233-651-0735 698-492-8776 x96552 320-691-2265 (744) 944-4122 x345 1-615-818-6954 x2101 1-405-466-8258 x8935 1-714-880-5597 (222) 862-3383 x37668 (620) 458-9058 x345 (492) 674-4980 x94724 644.314.7533 x444 1-315-792-0441 x838 
 713-609-1221 843-530-0629 x0379 300.398.9856 (219) 799-6469 x4611 1-974-633-5886 x421 (592) 618-2647 x77463 1-356-297-7748 x1684 882-937-0290 719-711-7382 x8312 1-849-292-6738 1-947-500-5759 x46315 (711) 492-2868 1-587-631-3274 x2526 1-852-240-8205 x060 360.942.8199 x6677 
 1-708-342-0933 x1129 (733) 724-6907 (756) 797-8144 x5264 (293) 674-3075 x9344 (941) 284-8291 1-937-674-1980 x970 (233) 692-4358 801.891.1711 x34294 (739) 448-0208 x05980 884.387.5957 x3988 548-913-7718 x3578 1-354-627-0149 1-542-648-5721 x5377 (201) 936-2365 x77371 1-374-685-4614 x16701 
 698-559-4879 x8283 967-205-9369 x004 1-395-800-6227 x744 1-553-647-1085 1-252-875-4180 444-465-2649 460.587.8477 x9978 569-509-9121 x550 1-902-275-4877 590.426.5394 855.623.8442 x9674 418-752-4179 731-384-7323 1-529-562-1049 642-513-0877 x059 
 435-518-6993 x89438 517.406.0291 (830) 860-6287 1-818-446-4359 x806 877-803-8196 x5196 1-626-689-9003 846.562.4103 x4837 (649) 936-7985 x394 788.425.1848 x875 (339) 979-0978 x39622 320-605-3408 961-745-2612 x2230 1-617-533-4224 x1796 1-597-276-6848 (459) 210-4668 x92076 AddressRows 
 93902 Nikolaus Shoals 5938 Adela Causeway 006 Boris Circles 360 Fay Plains 9276 Kameron Causeway 3346 McLaughlin Center 5555 Karolann Plaza 5766 Daugherty Dam 557 Emard Estate 24159 Schowalter Forges 4564 Haleigh Port 099 Dee Landing 423 Gregoria Pike 539 Keeling Junction 69767 Fritsch Crossing 
 714 Murray Square 99869 Hintz Street 9807 Kuhic Mission 67939 Palma Estates 61475 Einar Drives 838 Becker Pines 7798 Klocko Drive 463 Bahringer Station 9915 Abshire Terrace 403 Albertha Station 4790 Daren Spurs 970 Predovic Brooks 35111 D'Amore Tunnel 5643 Hahn Brooks 21744 Schinner Rapid 
 682 Johnston Mission 7305 Alberto Ports 62605 Gracie Gateway 31029 Olin Burgs 85259 Jaron Fall 41437 Gerson Spring 8899 Nitzsche Ways 7457 Grimes Summit 8847 Flatley Lodge 994 Leda Course 0920 Jonathan Roads 5958 McGlynn Parkway 655 Colleen Brook 3568 Lehner Glens 5804 Stamm Avenue 
 032 Rutherford Ford 45815 Amalia Hills 71657 Erdman Glen 982 Ruthie Falls 548 Ebert Crest 500 Monty Plains 6699 Leffler Mews 469 Casper Radial 072 Faustino Extensions 46383 Hermiston Manors 042 Balistreri Fall 2148 Hansen Square 3071 Torp Drive 43812 Newell Field 7309 Grant Rapid 
 62009 Major Terrace 06041 Emmerich Cape 28062 Raynor Cape 485 Williamson Ridges 33366 Rippin Trafficway 83539 Caleb Stream 7965 Pollich Spurs 371 Ashton Square 2248 Aimee Tunnel 079 Zemlak Circles 706 Mikayla Plains 606 Lyla Track 73689 Laron Orchard 84201 Shayne Summit 4874 Cartwright Union EmployeeCount 22 18 21 16 16 Employees Employees 
 1/5 2/5 3/5 4/5 5/5 Id 1000 1001 1002 1003 1004 Name Randal Reynolds Torey Thompson Rae Nicolas Amos Flatley Constance Price Title National Infrastructure Architect Customer Implementation Liaison Future Division Administrator Lead Configuration Administrator Chief Functionality Analyst Email Elna.Schmidt@hotmail.com Kristoffer_Prosacco@hotmail.com Ernestine_Runolfsdottir58@gmail.com Coralie.Sipes@yahoo.com Aryanna15@hotmail.com Telephone 1-986-593-4596 x6178 (450) 225-2563 509-750-8320 338.694.1351 696-782-5861 x5483 Employees 
 1/5 2/5 3/5 4/5 5/5 Id 1022 1023 1024 1025 1026 Name Maximillia Kuhlman Reyes Runolfsdottir Evan Ryan Maxie Spencer Cheyenne Abernathy Title Direct Optimization Architect Future Brand Liaison Forward Applications Strategist Investor Security Assistant Internal Response Engineer Email Flo.Wisozk44@gmail.com Jazmyn_Zboncak27@hotmail.com Eliane.Crona@yahoo.